In [16]:
import gmsh
import meshio
import os

# --- Geometry parameters ---
L = 5.0   # length
H = 1.0   # height
h = 0.1   # target mesh size

gmsh.initialize()
gmsh.model.add("Cantilever2D")

# --- Geometry: rectangle from (0,0) to (L,H) ---
p1 = gmsh.model.occ.addPoint(0, 0, 0, h)
p2 = gmsh.model.occ.addPoint(L, 0, 0, h)
p3 = gmsh.model.occ.addPoint(L, H, 0, h)
p4 = gmsh.model.occ.addPoint(0, H, 0, h)

l1 = gmsh.model.occ.addLine(p1, p2)
l2 = gmsh.model.occ.addLine(p2, p3)
l3 = gmsh.model.occ.addLine(p3, p4)
l4 = gmsh.model.occ.addLine(p4, p1)

cl = gmsh.model.occ.addCurveLoop([l1, l2, l3, l4])
surf = gmsh.model.occ.addPlaneSurface([cl])

gmsh.model.occ.synchronize()

# --- Physical groups ---
gmsh.model.addPhysicalGroup(2, [surf], tag=1)
gmsh.model.setPhysicalName(2, 1, "BeamDomain")

# Left edge (x=0)
gmsh.model.addPhysicalGroup(1, [l4], tag=2)
gmsh.model.setPhysicalName(1, 2, "LeftSupport")

# All edges (optional)
gmsh.model.addPhysicalGroup(1, [l1, l2, l3, l4], tag=3)
gmsh.model.setPhysicalName(1, 3, "AllEdges")

# Load point (top-right corner)
gmsh.model.addPhysicalGroup(0, [p3], tag=4)
gmsh.model.setPhysicalName(0, 4, "LoadPoint")

# --- Mesh generation ---
gmsh.model.mesh.generate(2)
gmsh.write("beam2d.msh")
gmsh.finalize()

print("✅ Gmsh mesh generated: beam2d.msh")


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000317584s, CPU 0.000383s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0216118s, CPU 0.02018s)
Info    : 663 nodes 1328 elements
Info    : Writing 'beam2d.msh'...
Info    : Done writing 'beam2d.msh'
✅ Gmsh mesh generated: beam2d.msh


In [17]:

# --- Convert to XDMF for FEniCS 2019 ---
msh = meshio.read("beam2d.msh")

# Separate out cell types
triangle_cells = []
line_cells = []
point_cells = []


In [18]:
msh.cell_data

{'vertex': {'gmsh:physical': array([4])},
 'line': {'gmsh:physical': array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         2, 2, 2, 2, 2, 2, 2, 2, 2, 2])},
 'triangle': {'gmsh:physical': array([1, 1, 1, ..., 1, 1, 1])}}

In [19]:

if "triangle" in msh.cells.keys():
    triangle_cells = {"triangle": msh.cells["triangle"]}
if "line" in msh.cells.keys():
    line_cells = {"line": msh.cells["line"]}
if "vertex" in msh.cells.keys():
    point_cells = {"vertex": msh.cells["vertex"]}

In [20]:

# Cell data mapping
cell_data = msh.cell_data

triangle_data = [cell_data["triangle"]["gmsh:physical"]] if "triangle" in cell_data else []
line_data = [cell_data["line"]["gmsh:physical"]] if "line" in cell_data else []
point_data = [cell_data["vertex"]["gmsh:physical"]] if "vertex" in cell_data else []


In [21]:
pts = msh.points[:,:2]

In [22]:
# Write domain mesh
meshio.write(
    "mesh.xdmf",
    meshio.Mesh(points=pts, cells=triangle_cells,
                cell_data={"triangle": {"name_to_read": triangle_data[0]}})
)

# Write facet markers
if line_cells:
    meshio.write(
        "facet_markers.xdmf",
        meshio.Mesh(points=pts, cells=line_cells,
                    cell_data={"line": {"name_to_read": line_data[0]}})
    )

# Write point markers (for load point)
if point_cells:
    meshio.write(
        "point_markers.xdmf",
        meshio.Mesh(points=pts, cells=point_cells,
                    cell_data={"vertex": {"name_to_read": point_data[0]}})
    )

print("✅ XDMF meshes written:")

✅ XDMF meshes written:
